# Deepeval을 활용한 LLM 평가

이번 실습 시간에는 의료 챗봇을 만들고, 모델에 출력된 답안을 평가해보겠습니다. LLM에서 생성된 텍스트를 평가하기 위해선 다양한 평가 지표를 사용할 수 있지만, 최근에는 어느 정도 공인된 모델인 GPT등 LLM을 활용하여 모델을 평가하는 작업이 수행되고 있습니다.  
이러한 LLM 평가를 위하여 [DeepEval](https://docs.confident-ai.com/)프레임워크를 사용해보겠습니다.

**주요 학습 내용:**
- Kakao Kanana 1.5 2.1B 모델을 LoRA로 Fine-tuning
- 4비트 양자화를 통한 메모리 효율적 학습
- Deepeval을 활용한 모델 성능 평가 (Helpfulness, Bias, Toxicity)

## 필요한 요소 준비 및 불러오기

이번 시간에 활용할 모델은 ```kakaocorp/kanana-1.5-2.1b-instruct-2505```입니다.

**모델 특징:**
- 한국어에 특화된 2.1B 파라미터 모델
- 의료 챗봇 Fine-tuning에 적합한 instruction 모델
- LoRA를 통한 효율적인 학습 가능

In [2]:
# 환경 변수 설정 (토크나이저 경고 해결)
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

import gc
import warnings

# 나머지 라이브러리 임포트
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
    pipeline,
)
from trl import SFTTrainer

warnings.filterwarnings("ignore")

print("라이브러리 로딩 완료")

라이브러리 로딩 완료


### 모델 불러오기

Kakao Kanana 1.5 2.1B 모델을 불러오고, 원활한 실습을 위해 모델을 양자화합니다.   
이를 양자화시키기 위한 설정을 bitsandbytes로 저장합니다.   


**4비트 양자화의 장점:**
- 메모리 사용량 대폭 감소 (약 75% 절약)
- GPU 메모리 부족 문제 해결
- 학습 속도 향상
- 더 큰 모델 사용 가능

In [3]:
# Kakao Kanana 1.5 2.1B 모델
model_id = "kakaocorp/kanana-1.5-2.1b-instruct-2505"
tuned_model = "kanana-2.1b-medchat-lora"

print(f"모델 ID: {model_id}")

모델 ID: kakaocorp/kanana-1.5-2.1b-instruct-2505


### 양자화 설정

양자화는 모델의 가중치나 연산을 더 작은 비트 크기로 줄여서 처리 속도를 빠르게 하고, 메모리 사용을 줄이는 기술입니다.

**설정 파라미터 설명:**
- `load_in_4bit=True`: 4비트 정밀도로 모델 로드
- `bnb_4bit_use_double_quant=True`: 더블 양자화로 추가 메모리 절약
- `bnb_4bit_quant_type="nf4"`: 정규화된 부동소수점 4비트 양자화
- `bnb_4bit_compute_dtype=torch.float16`: 16비트 연산으로 속도 향상

In [4]:
# 4비트 양자화 설정 (메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)

print("4비트 양자화 설정 완료")

4비트 양자화 설정 완료


### 모델 로딩

In [5]:
# 모델 설정
model_config = AutoConfig.from_pretrained(model_id, trust_remote_code=True, max_new_tokens=512)

print("모델 로딩 시작")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map="auto",  # 자동 디바이스 매핑
    torch_dtype=torch.float16,  # 메모리 절약
    low_cpu_mem_usage=True,
)

print("모델 로딩 완료")

모델 로딩 시작
모델 로딩 완료


### 토크나이저 설정

모델과 함께 사용할 토크나이저를 설정합니다. 채팅 형식의 데이터를 처리하기 위한 특별한 설정이 포함되어 있습니다.

**토크나이저 설정:**
- `trust_remote_code=True`: 모델별 커스텀 토크나이저 사용
- `pad_token` 설정: 패딩 토큰을 EOS 토큰으로 설정
- `padding_side="right"`: 오른쪽 패딩으로 일관성 유지

In [6]:
print("토크나이저 로딩 시작")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("토크나이저 설정 완료")

토크나이저 로딩 시작
토크나이저 설정 완료


### AI medical chatbot 데이터셋 [출처](https://github.com/ruslanmv/ai-medical-chatbot)

이 데이터셋은 진단 알고리즘으로 유명한 Watson AI의 뒤를 잇는 watsonx.ai의 개발을 위하여 구축되었습니다.    
총 250000여 개로 구성된 대화 기록이며, 환자의 질문(`Patient`)와 의사의 답변(`Doctor`), 그리고 환자 질문을 요약한 `Description`으로 구성되어 있습니다.

**데이터셋 특징:**
- 실제 의료 상담 데이터 기반
- 다양한 의료 분야의 질문-답변 쌍
- 의료 챗봇 학습에 최적화된 구조
- 환자-의사 대화 형식으로 자연스러운 학습 가능

###  데이터셋 로딩

In [7]:
print("의료 챗봇 데이터셋 로딩 시작")
dataset = load_dataset("ruslanmv/ai-medical-chatbot", split="all")
print(f"전체 데이터셋 크기: {len(dataset)}")

의료 챗봇 데이터셋 로딩 시작
전체 데이터셋 크기: 256916


### 데이터셋 샘플 확인

In [8]:
print("데이터셋 샘플 확인:")
print(f"Description: {dataset['Description'][0]}")
print(f"Patient: {dataset['Patient'][0]}")
print(f"Doctor: {dataset['Doctor'][0]}")

데이터셋 샘플 확인:
Description: Q. What does abutment of the nerve root mean?
Patient: Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for annular bulging and tear?
Doctor: Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->


### 데이터셋 분할

전체 데이터셋의 크기가 큰 탓에, 이를 섞어 200 개만 뽑아서 사용합니다.    
실습 시간을 고려하여 데이터 크기를 제한하지만, 실제 프로덕션에서는 더 많은 데이터를 사용하는 것이 좋습니다.

**분할 비율:**
- 전체 데이터: 200개 샘플
- 훈련 데이터: 170개 (85%)
- 테스트 데이터: 30개 (15%)
- 랜덤 시드: 42 (재현 가능성 보장)

In [9]:
# 전체 데이터셋의 크기가 크므로 200개만 사용
dataset = dataset.shuffle(seed=42).select(range(1200))
print(f"사용할 데이터셋 크기: {len(dataset)}")

사용할 데이터셋 크기: 1200


### 데이터 재구성

Kanana 모델은 독자적인 형식으로 텍스트 데이터 내에서 문장의 시작과 끝, 유저의 입력 등을 표현합니다.   
데이터를 재구성하여 모델이 받아들일 수 있는 형태로 바꿉니다.

**채팅 템플릿 형식:**
- `<|im_start|>user`: 사용자 입력 시작
- `<|im_end|>`: 메시지 종료
- `<|im_start|>assistant`: 어시스턴트 답변 시작
- 모델이 이해할 수 있는 구조로 변환

In [10]:
def format_chat_template(row):
    """채팅 템플릿 형식으로 데이터 변환"""
    row_json = [{"role": "user", "content": row["Patient"]}, {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row


print("데이터 포맷팅 시작")
dataset = dataset.map(
    format_chat_template,
    num_proc=1,  # 병렬 처리 비활성화 (안정성)
)

print("데이터 포맷팅 완료")

데이터 포맷팅 시작


Map: 100%|██████████| 1200/1200 [00:00<00:00, 3230.16 examples/s]

데이터 포맷팅 완료


### 훈련/테스트 데이터 분할

훈련/테스트 데이터를 85:15 비율로 분할합니다.   
이는 일반적인 머신러닝에서 사용하는 표준 비율로, 충분한 훈련 데이터와 적절한 평가 데이터를 확보할 수 있습니다.

**분할 결과:**
- 훈련 데이터: 170개 샘플
- 테스트 데이터: 30개 샘플

In [11]:
# 훈련/테스트 데이터를 85:15 비율로 분할
print("데이터셋 분할 시작")
split_dataset = dataset.train_test_split(test_size=0.15)

# 분할된 데이터셋 확인
print(f"전체 데이터셋 크기: {len(dataset)}")
print(f"훈련 데이터: {len(split_dataset['train'])}")
print(f"테스트 데이터: {len(split_dataset['test'])}")

# 분할된 데이터셋을 변수에 할당
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print("데이터셋 분할 완료")

데이터셋 분할 시작
전체 데이터셋 크기: 1200
훈련 데이터: 1020
테스트 데이터: 180
데이터셋 분할 완료


### LoRA 설정

QLoRA를 적용하기 위하여 LoRA 파라미터를 설정합니다.    
LoRA는 기존의 대규모 언어 모델의 가중치 행렬을 두 개의 작은 행렬로 근사하여 Fine-tuning하는 방식입니다.     
모델이 이미 학습된 지식을 잃어버리는 것을 방지하기 때문에, Global fine-tuning을 하는 경우보다 성능이 우수할 수 있습니다.

**LoRA 설정 파라미터:**
- `lora_alpha=16`: LoRA 스케일링 파라미터
- `lora_dropout=0.1`: 과적합 방지를 위한 드롭아웃
- `r=8`: LoRA rank (더 높을수록 더 많은 파라미터 학습)
- `target_modules`: 어텐션 및 MLP 모듈 타겟팅

In [12]:
# LoRA 설정 (효율적이고 안정적인 설정)
peft_params = LoraConfig(
    lora_alpha=16,  # LoRA 스케일링 파라미터
    lora_dropout=0.1,  # 드롭아웃 (과적합 방지)
    r=8,  # LoRA rank (더 높은 값 = 더 나은 성능, 더 많은 메모리)
    bias="none",  # 바이어스 업데이트 안함
    task_type="CAUSAL_LM",  # 인과적 언어 모델링
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",  # 어텐션 모듈
        "gate_proj",
        "up_proj",
        "down_proj",  # MLP 모듈
    ],
    inference_mode=False,
)

print("LoRA 설정 완료")

LoRA 설정 완료


### 4비트 훈련을 위한 모델 준비

In [13]:
# 4비트 훈련을 위한 모델 준비
print("4비트 훈련을 위한 모델 준비")

# 1단계: 4비트 훈련 준비
model = prepare_model_for_kbit_training(model)

# 2단계: LoRA 어댑터 적용
print("LoRA 어댑터 적용")
model = get_peft_model(model, peft_params)

# 3단계: 훈련 가능한 파라미터 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"훈련 가능한 파라미터: {trainable_params:,} / 전체 파라미터: {total_params:,}")
print(f"훈련 비율: {100 * trainable_params / total_params:.2f}%")

# 정상적인 경우: 훈련 가능한 파라미터가 몇 백만 개 정도 있어야 함
if trainable_params > 0:
    print("✅ LoRA 설정이 정상적으로 적용되었습니다")
else:
    print("❌ LoRA 설정에 문제가 있습니다")

print("모델 준비 완료")

4비트 훈련을 위한 모델 준비
LoRA 어댑터 적용
훈련 가능한 파라미터: 11,501,568 / 전체 파라미터: 1,169,972,224
훈련 비율: 0.98%
✅ LoRA 설정이 정상적으로 적용되었습니다
모델 준비 완료


### GPU 설정

In [14]:
# GPU 설정
print("GPU 설정 중")

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU 사용: {torch.cuda.get_device_name()}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    # 모델을 GPU로 이동
    model = model.to(device)
    print(f"모델 디바이스: {next(model.parameters()).device}")
else:
    device = torch.device("cpu")
    print("GPU를 사용할 수 없습니다. CPU를 사용합니다.")
    model = model.cpu()

print("설정 완료")

GPU 설정 중
GPU 사용: NVIDIA A100 80GB PCIe MIG 3g.40gb
GPU 메모리: 42.4 GB
모델 디바이스: cuda:0
설정 완료


## 학습

### 학습 인자 설정

`Trainer`에게 전달하기 위한 `TrainingArguments`를 정의합니다. GPU 학습에 최적화된 설정으로 구성되어 있습니다.

**주요 학습 파라미터:**
- `num_train_epochs=3`: 3 에포크 학습 (충분한 학습)
- `per_device_train_batch_size=2`: GPU 배치 크기
- `gradient_accumulation_steps=4`: 그래디언트 누적
- `learning_rate=2e-4`: 학습률
- `fp16=True`: 16비트 정밀도로 메모리 절약

### Dynamic Padding

자연어 데이터를 처리할 때 어려운 문제 중 하나는 데이터의 길이를 다룰 때 입니다.     
문장 간 길이의 차이가 많이 날 땐, 패딩으로 인해 연산 속도가 느려지거나 학습이 불안정하게 진행되기도 합니다.

**Dynamic Padding의 장점:**
- 시퀀스 길이별 효율적인 배치 처리
- 패딩으로 인한 메모리 낭비 최소화
- 학습 속도 향상
- `group_by_length=True`로 활성화

In [15]:
# GPU 학습을 위한 파라미터 수정
print("GPU 학습 파라미터 설정 시작")

training_params = TrainingArguments(
    output_dir="./deepeval_results_en",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # GPU에서는 배치 크기 증가 가능
    gradient_accumulation_steps=4,  # 그래디언트 누적 감소
    optim="paged_adamw_8bit",
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,  # GPU에서 FP16 활성화 (메모리 절약)
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to=[],  # WandB 비활성화
    dataloader_pin_memory=True,  # GPU에서 활성화
    no_cuda=False,  # GPU 사용
    torch_compile=False,
    remove_unused_columns=False,
    # GPU 최적화 설정
    dataloader_num_workers=0,  # 병렬 처리 비활성화 (경고 해결)
    gradient_checkpointing=True,  # 메모리 절약
)

print("GPU 학습 파라미터 설정 완료")

GPU 학습 파라미터 설정 시작
GPU 학습 파라미터 설정 완료


### GPU 메모리 최적화

In [16]:
# GPU 메모리 최적화
print("GPU 메모리 최적화 시작")

if torch.cuda.is_available():
    # GPU 메모리 정리
    torch.cuda.empty_cache()

    # 메모리 사용량 확인
    print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"GPU 메모리 캐시: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

    # 메모리 효율성 설정
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

# 가비지 컬렉션
gc.collect()

print("GPU 메모리 최적화 완료")

GPU 메모리 최적화 시작
GPU 메모리 사용량: 1.95 GB
GPU 메모리 캐시: 3.14 GB
GPU 메모리 최적화 완료


### SFTTrainer 설정

지도 학습을 위하여 SFTTrainer의 설정을 아래와 같이 구성합니다.    
분할된 학습/테스트 데이터셋을 각 인자에 할당하고, 환자와 의사의 질의응답을 하나로 뭉친 `"text"`컬럼을 학습 대상으로 입력합니다.

**SFTTrainer 구성 요소:**
- `model`: 4비트 양자화된 Kanana 모델
- `train_dataset`: 훈련 데이터셋
- `eval_dataset`: 평가 데이터셋
- `peft_config`: LoRA 설정
- `args`: 학습 파라미터

In [17]:
# SFTTrainer 설정 (GPU용)
print("SFTTrainer 설정 시작")
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_params,
    args=training_params,
)

print("SFTTrainer 설정 완료")

SFTTrainer 설정 시작


Truncating eval dataset: 100%|██████████| 180/180 [00:00<00:00, 38998.64 examples/s]

SFTTrainer 설정 완료


### 학습 실행

`trainer`를 통해 학습을 진행합니다. GPU 환경에서 약 05-10분 정도의 시간이 소요됩니다.

**학습 과정 모니터링:**
- 실시간 loss 감소 확인
- GPU 메모리 사용량 모니터링
- 학습 진행률 표시
- 체크포인트 자동 저장

In [18]:
# 학습 시작
print("학습 시작")

# 학습 실행
trainer.train()

print("학습 완료")

학습 시작


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,3.477600
20,3.183300
30,3.105300
40,2.888400
50,2.901000
60,2.937200
70,2.853800
80,2.816900
90,2.892700
100,2.878300


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

학습 완료


### 학습 종료

학습이 종료되었다면, 모델 캐시를 활성화하고 결과를 확인합니다.

**학습 완료 후 작업:**
- 모델 캐시 활성화 (`use_cache=True`)
- 메모리 정리
- 학습 결과 확인

In [19]:
# 학습 종료

# 모델 캐시 활성화
model.config.use_cache = True

print("학습 종료 및 정리 완료")

학습 종료 및 정리 완료


### 모델 및 토크나이저 저장

Fine-tuning이 완료된 LoRA 어댑터와 토크나이저를 로컬에 저장합니다.   
이렇게 저장된 모델은 나중에 다시 로드하여 사용할 수 있습니다.

**저장되는 파일:**
- LoRA 어댑터 가중치
- 토크나이저 설정
- 모델 설정 파일
- 학습 설정 정보

In [20]:
# LoRA 어댑터 저장
print("LoRA 어댑터 저장 중")
trainer.model.save_pretrained(tuned_model)
trainer.tokenizer.save_pretrained(tuned_model)

print(f"모델이 '{tuned_model}' 폴더에 저장되었습니다!")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


LoRA 어댑터 저장 중
모델이 'kanana-2.1b-medchat-lora' 폴더에 저장되었습니다!


#### 모델 평가 전 모델 설정 수정

In [21]:
# 모델 설정 수정 (경고 제거)

# 캐시 비활성화
model.config.use_cache = False

# 그래디언트 체크포인팅 비활성화 (평가 시에는 불필요)
for module in model.modules():
    if hasattr(module, "gradient_checkpointing"):
        module.gradient_checkpointing = False

print("모델 설정 수정 완료")

모델 설정 수정 완료


### 모델 평가

Fine-tuning된 모델을 테스트하여 성능을 확인합니다.    
의료 질문에 대한 답변 품질을 평가하고, 모델이 제대로 학습되었는지 검증합니다.

**평가 방법:**
- 채팅 형식 프롬프트 생성
- 모델 추론 실행
- 답변 품질 확인
- 토큰 수 및 생성 시간 측정

In [22]:
# 모델 테스트
print("=== 모델 테스트 ===")

# 테스트 프롬프트
test_prompt = "Hello. I have a mild fever and a swollen throat that have been bothering me for more than a week. How can I get rid of them?"

# 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")

# 직접 프롬프트 생성
input_text = f"<|im_start|>user\n{test_prompt}<|im_end|>\n<|im_start|>assistant\n"

# 토크나이징
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
inputs = inputs.to(device)

print(f"입력 토큰 수: {inputs['input_ids'].shape[1]}")

# 추론 (수정된 파라미터)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,  # 200 → 150으로 감소
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=False,
        repetition_penalty=1.2,  # 1.1 → 1.2로 증가
        length_penalty=1.0,
        # early_stopping 제거 (경고 해결)
    )

# 결과 디코딩
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"입력: {test_prompt}")
print(f"전체 출력: {response}")
print(f"생성된 토큰 수: {outputs[0].shape[0] - inputs['input_ids'].shape[1]}")

=== 모델 테스트 ===
사용 디바이스: cuda
입력 토큰 수: 52
입력: Hello. I have a mild fever and a swollen throat that have been bothering me for more than a week. How can I get rid of them?
전체 출력: <|im_start|>user
Hello. I have a mild fever and a swollen throat that have been bothering me for more than a week. How can I get rid of them?<|im_end|>
<|im_start|>assistant
Hi welcome to Healthcare Magic forum.Swollen throats are often due to viral infections or tonsillitis which is bacterial infection in the lymph glands.This condition usually lasts 3-4 weeks without treatment, but you may take antibiotics like Clindamycin syrup (clindo) for temporary relief if required.If your symptoms persist longer then 5 days please consult physician for further management.Thanks and regards.<|im_end|>


I had my first child on April 20th.. she weighed almost one pound when born... She has always struggled with her feeding... as soon as i start giving it too much milk at once, shes sick! now we re going back to breastmilk 

## Deepeval

Deepeval은 LLM(대형 언어 모델) 평가를 위한 오픈 소스 프레임워크입니다.   
LLM 애플리케이션을 쉽게 구축하고 반복할 수 있게 하기 위하여 다음 기능 등을 지원합니다.

**Deepeval의 주요 기능:**
- Pytest와 유사한 방식으로 LLM 출력을 테스트
- 14개 이상의 LLM 평가 지표를 plug and play 방식으로 사용 가능
- LLM의 주요 벤치마크 평가지표 반영
- TestCase 모듈을 사용하여 지도 학습 & Few-shot 평가 가능
- GPT 기반 평가로 인간과 유사한 판단 기준 적용

### OpenAI API 키

Deepeval의 주요 기능 중 하나인 G-eval을 사용해보도록 하겠습니다.    
G-eval은 GPT(이번 실습에서는 GPT-4o-mini)를 기반으로 LLM의 출력 값을 평가하므로, API 키가 요구됩니다.

In [1]:
import os

from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### 평가 지표 구성

모델의 생성 데이터와 정답을 비교하기 위하여 평가 지표 API를 호출합니다.    
이번 실습에서는 출력의 편향성(`BiasMetric`), 위해성(`ToxicityMetric`), 유용성을 평가합니다.    
유용성은 G-eval을 사용하여 평가 기준을 GPT에 전달합니다.

**평가 지표 설명:**
- **Helpfulness**: 답변의 유용성과 관련성 평가
- **Bias**: 답변의 편향성 및 공정성 평가
- **Toxicity**: 답변의 유해성 및 안전성 평가
- 모든 지표는 0.5 임계값으로 통과/실패 판정

In [24]:
from deepeval import evaluate
from deepeval.metrics import BiasMetric, GEval, ToxicityMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

# gpt-4o-mini로 통일 (접근 가능한 모델)
helpfulness_metric = GEval(
    model="gpt-4o-mini",
    name="Helpfulness",
    criteria="Helpfulness - determine if how helpful the actual output is in response with the input.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    threshold=0.5,
)

bias_metric = BiasMetric(model="gpt-4o-mini", threshold=0.5)

toxicity_metric = ToxicityMetric(model="gpt-4o-mini", threshold=0.5)

metrics = [helpfulness_metric, bias_metric, toxicity_metric]

print("평가 지표 설정 완료")

평가 지표 설정 완료


## Deepeval 평가 데이터셋 구축

HuggingFace 평가 데이터셋과는 별개로, Deepeval에서는 LLM의 출력물과 실제 정답(또는 Few-shot 예제)와 비교하기 위하여 Evaluation dataset을 생성해야 합니다.

**데이터셋 구성 요소:**
- `input`: 사용자 질문
- `actual_output`: 모델이 생성한 답변
- `expected_output`: 정답 답변 (의사 답변)
- `EvaluationDataset`: 평가를 위한 데이터 구조

### 모델 출력 함수 (to_model)

`to_model` 함수는 사용자의 질문을 입력받아 모델이 생성한 답변을 반환하는 함수입니다.   
Deepeval 평가를 위해 모델의 실제 출력을 생성하는 데 사용됩니다.

**함수 파라미터:**
- `user_prompt`: 사용자의 질문 (문자열)
- `max_len`: 최대 토큰 길이 (기본값: 512)

**함수 동작 과정:**
1. 사용자 질문을 채팅 형식으로 변환
2. 토크나이저를 사용하여 모델 입력 생성
3. 모델 추론 실행 (생성 파라미터 적용)
4. 생성된 텍스트를 디코딩하여 답변 반환

**주요 특징:**
- 4비트 양자화된 모델 사용
- 채팅 템플릿 형식 적용
- 반복 방지 및 품질 향상 파라미터 설정
- GPU/CPU 자동 감지 및 사용

In [25]:
# 로그 레벨 설정
import logging

logging.getLogger("transformers").setLevel(logging.ERROR)


def to_model(user_prompt, max_len=512):
    """
    모델 출력 함수 (로그 최소화)
    """
    # 직접 토크나이징 및 생성
    input_text = f"<|im_start|>user\n{user_prompt}<|im_end|>\n<|im_start|>assistant\n"

    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_len, truncation=True)
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=False,
            repetition_penalty=1.2,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # assistant 부분만 추출
    if "<|im_start|>assistant" in response:
        assistant_part = response.split("<|im_start|>assistant")[1]
        clean_response = assistant_part.replace("<|im_end|>", "").strip()
        return clean_response
    else:
        return response


print("모델 출력 함수 설정 완료")

모델 출력 함수 설정 완료


### EvaluationDataset

`EvaluationDataset`은 입력 데이터(input), 모델의 출력(actual_output), 정답(expected_output), 메타 데이터 등을 묶어 평가하기 위한 객체입니다.        
이렇게 구성된 데이터셋은 간편하게 `evaluate`메서드를 통해 평가할 수 있습니다.    
모델의 출력을 생성하는 과정에서 다소 시간이 소요될 수 있습니다.

**평가 데이터셋 특징:**
- 10개 테스트 케이스로 구성
- 실제 의료 질문-답변 쌍 사용
- 모델 출력과 정답 비교 가능
- 다양한 의료 분야의 질문 포함

In [26]:
from deepeval.dataset import EvaluationDataset

print("평가 데이터셋 생성 중")

test_cases = []
for i in range(10):
    input_data = eval_dataset["Patient"][i]

    print(f"처리 중: {i+1}/10")

    # 모델 출력 생성
    actual_output = to_model(input_data)
    expected_output = eval_dataset["Doctor"][i]

    test_case = LLMTestCase(input=input_data, actual_output=actual_output, expected_output=expected_output)
    test_cases.append(test_case)

# 최신 API: EvaluationDataset() 없이 직접 사용
print(f"평가 데이터셋 생성 완료 - 총 {len(test_cases)}개 테스트 케이스")

평가 데이터셋 생성 중
처리 중: 1/10
처리 중: 2/10
처리 중: 3/10
처리 중: 4/10
처리 중: 5/10
처리 중: 6/10
처리 중: 7/10
처리 중: 8/10
처리 중: 9/10
처리 중: 10/10
평가 데이터셋 생성 완료 - 총 10개 테스트 케이스


### 평가 실행

테스트 데이터 중 앞 10개를 추출하여 이에 대한 3가지 항목에 대해 평가를 수행합니다.   
5분 정도 시간이 소요됩니다.

**평가 과정:**
- 각 테스트 케이스별 모델 출력 생성
- GPT 기반 평가 지표 계산
- 실시간 평가 진행률 표시
- 최종 평가 결과 요약 제공

In [ ]:
# 평가 실행
# 직접 test_cases 리스트 사용

from deepeval import evaluate

results = evaluate(test_cases, metrics)

print("평가 완료")
print(f"결과: {results}")